In [1]:

# 2018年秋季大会-2022年秋季大会の全ての発表（物性系）について、著者名を元にプログラムの検索を利用してjsonl化する。素核用
import numpy as np


meetings, fromdates, todates = np.genfromtxt('https://raw.githubusercontent.com/minowayosuke/jps_presentation_researchmap/main/%E5%AD%A6%E4%BC%9A%E6%83%85%E5%A0%B1_%E7%B4%A0%E6%A0%B8.csv',delimiter=",",dtype='str',encoding='utf-8-sig',unpack =True)
authors = "山田太郎" #検索用の著者名
user_id = "B000000000" #researchmap会員ID



In [2]:
%%shell
# https://github.com/googlecolab/colabtools/issues/3347
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap
#

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium



Executing: /tmp/apt-key-gpghome.dxIzhy9BP4/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.vixZrXHCg0/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.gWj8nUEPAV/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/l

In [3]:

import time 
time.sleep(3)

In [4]:
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re

import copy
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

options.headless = True






myjsonl = []


mytemplate = {
  "insert": {
    "type": "presentations",
    "id": None,
    "user_id": None
  },
  "similar_merge": {
    "display": "disclosed",
    "major_achievement": False,
    "presentation_title": {
      "ja": "仮タイトル"
    },
    "presenters": {
      "ja": [
        {
          "name": "山田太郎"
        },
        {
          "name": "山田花子"
        },
        {
          "name": "山田二郎"
        }
      ]
    },
    "event": {
      "ja": "日本物理学会 2022年秋季大会"
    },
    "publication_date": "2022-09-14",
    "from_event_date": "2022-09-12",
    "to_event_date": "2022-09-15",
    "invited": False,
    "presentation_type": "oral_presentation"
  }
}

for i, meeting_name in enumerate(meetings):
    time.sleep(10)

    jps_url = meeting_name
    fromdate = fromdates[i]
    todate = todates[i]


    mytemplate["insert"]["user_id"]=user_id
    mytemplate["similar_merge"]["from_event_date"]=fromdate
    mytemplate["similar_merge"]["to_event_date"]=todate



    # Pass in the `Service` instance with the `service` keyword: 
    driver = webdriver.Chrome(service=Service("/usr/bin/chromedriver"), options=options)
    driver.get(jps_url)

    element = driver.find_element(By.XPATH, '//*[@id="main_form"]/div[1]/table/tbody/tr[4]/td/input[2]')
    element.send_keys(authors)

    #検索結果がない場合の例外処理
    try:
        element.send_keys(Keys.ENTER)
        element_table = driver.find_elements(By.XPATH, '//*[@id="result_data_table"]')
        #wait = WebDriverWait(driver, 30)
        #wait.until(EC.alert_is_present())
        #alert = driver.switch_to.alert
        #alert.accept()
        #print("Alert accepted 0")
    except Exception:
        pass
    else:   

        rows = element_table[0].find_elements(By.TAG_NAME, "tr")

        #著者名の上付き文字を消す
        for row in rows[1:]:#表の一行目は飛ばす
            cells = row.find_elements(By.TAG_NAME,'td') 
            sup_removes = cells[3].find_elements(By.TAG_NAME,'sup') 
            for sup_remove in sup_removes:
                driver.execute_script('arguments[0].remove()', sup_remove)

        #jsonl用データ準備
        for row in rows[1:]:
            cells = row.find_elements(By.TAG_NAME,'td')
            presentation_style="oral_presentation"
            if re.sub('^[0-9]*','',cells[0].text)[0]=="P":
                presentation_style="poster_presentation"
            mytemplate["similar_merge"]["presentation_type"]=presentation_style
            mytemplate["similar_merge"]["presentation_title"]["ja"]=cells[2].text
            names = cells[3].text
            mytemplate["similar_merge"]["presenters"]["ja"] = [{'name': x.strip()} for x in names.split(',')]
            mytemplate["similar_merge"]["event"]["ja"] = driver.title
            presentation_id = cells[0].text
            mytemplate["similar_merge"]["publication_date"]=todate[:-2]+(re.findall(r"\d+", presentation_id)[0]).zfill(2)
            myjsonl.append(copy.deepcopy(mytemplate))

    driver.close()


with open('jps_presentations.jsonl', mode='w', encoding='utf-8') as fout:
    for obj in myjsonl:
        json.dump(obj, fout, ensure_ascii=False)
        fout.write('\n')

<ipython-input-4-020df0c5e882>:20: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [5]:
from google.colab import files

files.download('/content/jps_presentations.jsonl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>